In [1]:
import os
import pandas as pd
import numpy as np
import validation_data_input
import plotly.express as px
import toml
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
input_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'input_configuration.toml'))
# model_dir = os.path.join(os.getcwd(), '..\\..\\..\\..\\')

In [2]:
person = validation_data_input.get_data('person')
hh = validation_data_input.get_data('household')

In [3]:
# parcel land use data
df_parcel = pd.read_csv(os.path.join(config['model_dir'],r'outputs\landuse\buffered_parcels.txt'),
                        delim_whitespace=True,
                        usecols=['parcelid','emptot_1','hh_1'])

In [4]:
# Load parcel geography lookups
parcel_geog = pd.read_sql_table(
    'parcel_'+input_config['base_year']+'_geography',
    'sqlite:///'+config['model_dir']+'/inputs/db/'+input_config['db_name'],
    columns=['ParcelID','CountyName','rg_proposed']
    )

In [5]:
hh = hh.merge(df_parcel, left_on='hhparcel', right_on='parcelid', how='left')

In [6]:
# Group income, hh density, and employment density into 4 groups
var_group = hh.loc[hh['source'] == 'model',['hhincome','emptot_1','hh_1']].quantile([.25, .50, .75])

# var_group

In [7]:
hh['hhincome_group'] = pd.cut(hh['hhincome'],bins=[-9999999.0] + var_group['hhincome'].tolist() + [9999999.0], labels=['low', 'medium', 'medium-high', 'high'])
# add hh density groups
hh['hh_density_group'] = pd.cut(hh['hh_1'],bins=[-9999999.0] + var_group['hh_1'].tolist() + [9999999.0], labels=['low', 'medium', 'medium-high', 'high'])
# add employment density groups
hh['emp_density_group'] = pd.cut(hh['emptot_1'],bins=[-9999999.0] + var_group['emptot_1'].tolist() + [9999999.0], labels=['low', 'medium', 'medium-high', 'high'])


In [8]:
person = person.merge(hh, on=['hhno'], how='left')

In [9]:
person = person.merge(parcel_geog, left_on='pwpcl', right_on='ParcelID', how='left')
person = person.merge(parcel_geog, left_on='hhparcel', right_on='ParcelID', how='left', suffixes=['_work', '_home'])

In [10]:
person.drop('source_x', axis=1, inplace=True)
person.rename(columns={'source_y': 'source'}, inplace=True)

In [11]:
df_workers = person.loc[person['pwtyp']>0, :].copy()

In [12]:
df = df_workers.loc[df_workers['pwaudist'] > 0, :].copy()
df['wt_dist'] = df['pwaudist']*df['psexpfac']
df = df.groupby(['source'])[['wt_dist','psexpfac']].sum().reset_index()
df['Mean Distance to Work by Car (pwaudist)'] = df['wt_dist']/df['psexpfac']
df[['source','Mean Distance to Work by Car (pwaudist)']]

,source,Mean Distance to Work by Car (pwaudist)
0,model,10.449554
1,survey,11.608149
2,survey (2017/2019),11.967041


In [13]:
df_workers.to_clipboard()

In [14]:

def plot_work_location(df:pd.DataFrame, var:str, title_cat:str, sub_name:str):
    df = df[df['pwaudist'] > 0].copy()
    df['wt_dist'] = df['pwaudist']*df['psexpfac']
    df_plot = df.groupby(['source',var])[['wt_dist','psexpfac']].sum().reset_index()
    df_plot['average_wt_pwaudist'] = df_plot['wt_dist']/df_plot['psexpfac']
    df_plot

    df_plot_ct = df.groupby(['source',var])[['wt_dist','psexpfac']].count().reset_index(). \
    rename(columns={'psexpfac':'sample count'})
    df_plot = df_plot.merge(df_plot_ct, on=['source',var])

    fig = px.bar(df_plot, x=var, y=['average_wt_pwaudist', 'source'], color="source",
                    barmode="group",
                     hover_data=['sample count'],
                    title="Work Distance by "+ title_cat)
    fig.for_each_annotation(lambda a: a.update(text = sub_name + "=<br>" + a.text.split("=")[-1]))
    fig.update_xaxes(title_text=sub_name)
    fig.update_layout(height=400, width=800, font=dict(size=11),
                      yaxis=dict(tickformat=".2f"))
    fig.for_each_yaxis(lambda a: a.update(tickformat = ".2f"))
    fig.show()

## Work Location by Workplace Geography

In [15]:
plot_work_location(df_workers, 'CountyName_work', 'Work County', 'County')

In [16]:
plot_work_location(df_workers, 'rg_proposed_work', 'Regional Geography', 'Regional Geography')

## Workplace by Home Geography

In [17]:
plot_work_location(df_workers, 'CountyName_home', 'Home County', 'County')

In [18]:
plot_work_location(df_workers, 'rg_proposed_home', 'Home Regional Geography', 'Regional Geography')

## Workplace Location by Person/Household Characteristics

In [19]:
plot_work_location(df_workers, 'pwtyp', 'Employment Type', 'employment type')

In [20]:
plot_work_location(df_workers, 'hhincome_group', 'Income', 'Income Group')

In [21]:
plot_work_location(df_workers, 'hh_density_group', 'Household Density', 'Household Density Group')

In [22]:
plot_work_location(df_workers, 'emp_density_group', 'Employment Density', 'Employment Density Group')

In [23]:
# distance to school bins from workplace_location.csv
df_workers['distance_to_work_bin'] = pd.cut(df_workers['pwaudist'], bins=[0,1,2,5,15,9999],
                                    labels=['0-1', '1-2', '2-5',
                                            '5-15', '15+'])
# Create bins: bins of 2 miles up to 60 miles
max_bin = 60
bin_size = 2
df_workers['d_work_bin_60mi'] = pd.cut(df_workers['pwaudist'], bins=np.arange(0, max_bin+bin_size, bin_size), labels=[str(i) for i in np.arange(0, max_bin, bin_size)])

## Distance Bins

In [24]:
def plot_distance(df:pd.DataFrame, group:str, title_name:str):

    count = df.loc[(df['source']=='model') & (df[group]),['distance_to_work_bin']].value_counts()
    # print(f"model person count =\n"
    #       f"{count.sort_values()}")

    # plot1
    df_plot = df.loc[(df[group]) & (df['distance_to_work_bin'] != float('nan'))].groupby(['source','distance_to_work_bin'])['psexpfac'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['psexpfac'].\
            apply(lambda x: 100 * x / float(x.sum()))

    df_plot_ct = df.loc[(df['distance_to_work_bin'] != float('nan'))].groupby(['source','distance_to_work_bin'])['psexpfac'].count().reset_index().rename(columns={'psexpfac':'sample count'})
    df_plot = df_plot.merge(df_plot_ct, on=['source','distance_to_work_bin'])

    # df_plot
    #
    fig1 = px.bar(df_plot, x='distance_to_work_bin', y="percentage", color="source", barmode="group",hover_data=['sample count'],
                  title=title_name)
    fig1.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig1.update_layout(height=400, width=700, font=dict(size=11))
    fig1.show()

    # # plot2
    df_plot = df.loc[(df[group]) & (df['d_work_bin_60mi'].notna())].groupby(['source','d_work_bin_60mi'])['psexpfac'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['psexpfac'].\
                apply(lambda x: 100 * x / float(x.sum()))
    # df_plot
    #
    fig2 = px.line(df_plot, x='d_work_bin_60mi', y="percentage", color="source",
                   title=title_name)
    fig2.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig2.update_layout(height=400, width=700, font=dict(size=11))
    fig2.show()

# distance to work
plot_distance(df_workers,'pwtyp',"worker: distance to work (miles)")

In [25]:
def plot_group(df:pd.DataFrame, group:str, title_name:str):
    df_plot = df_workers.loc[(df_workers['pptyp']) & (df_workers['distance_to_work_bin'] != float('nan'))].groupby(['source',group,'distance_to_work_bin'])['psexpfac'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',group], group_keys=False)['psexpfac']. \
        apply(lambda x: 100 * x / float(x.sum()))
    
    df_plot_ct = df_workers.loc[(df_workers['pptyp']) & (df_workers['distance_to_work_bin'] != float('nan'))].groupby(['source',group,'distance_to_work_bin'])['psexpfac'].count().reset_index().rename(columns={'psexpfac':'sample count'})
    df_plot = df_plot.merge(df_plot_ct, on=['source',group,'distance_to_work_bin'])

    fig = px.bar(df_plot, x="distance_to_work_bin", y="percentage", color="source",barmode="group",
                facet_col=group, facet_col_wrap=2, hover_data=['sample count'],
                title="Distance to work by "+title_name)
    fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig.update_layout(height=900, width=700, font=dict(size=11))
    fig.show()

plot_group(df_workers, 'hhincome_group', 'income group')

In [26]:
def plot_group_line(df:pd.DataFrame, group:str, title_name:str):
    df_plot = df_workers.loc[(df_workers['pptyp']) & (df_workers['d_work_bin_60mi'] != float('nan'))].groupby(['source',group,'d_work_bin_60mi'])['psexpfac'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',group], group_keys=False)['psexpfac']. \
        apply(lambda x: 100 * x / float(x.sum()))
    # df_plot

    fig2 = px.line(df_plot, x='d_work_bin_60mi', y="percentage", color="source",
                facet_col=group, facet_col_wrap=2,
                title="Distance to work by "+title_name)
    fig2.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig2.update_layout(height=400, width=700, font=dict(size=11))
    fig2.show()

plot_group_line(df_workers, 'hhincome_group', 'household income')

In [27]:
plot_group(df_workers, 'hh_density_group', 'density group')

In [28]:
plot_group_line(df_workers, 'hh_density_group', 'household density')

In [29]:
plot_group(df_workers, 'CountyName_work', 'work county')

In [30]:
plot_group_line(df_workers, 'CountyName_work', 'Work County')

In [31]:
plot_group(df_workers, 'rg_proposed_work', 'work geography')

In [32]:
plot_group_line(df_workers, 'rg_proposed_work', 'Work Geography')